In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")

# Load the key datasets
print("Loading F1 datasets...")

# Core tables for qualifying prediction
qualifying = pd.read_csv('../data/qualifying.csv')
races = pd.read_csv('../data/races.csv')
drivers = pd.read_csv('../data/drivers.csv')
constructors = pd.read_csv('../data/constructors.csv')
results = pd.read_csv('../data/results.csv')

print("Datasets loaded successfully!")
print(f"Qualifying data: {qualifying.shape}")
print(f"Races data: {races.shape}")
print(f"Drivers data: {drivers.shape}")
print(f"Constructors data: {constructors.shape}")
print(f"Results data: {results.shape}")

Matplotlib is building the font cache; this may take a moment.


Loading F1 datasets...
Datasets loaded successfully!
Qualifying data: (10774, 9)
Races data: (1149, 18)
Drivers data: (864, 9)
Constructors data: (212, 5)
Results data: (27038, 18)


In [3]:
# Let's examine the qualifying data structure
print("=== QUALIFYING DATA EXPLORATION ===")
print("\nColumn names:")
print(qualifying.columns.tolist())

print("\nFirst few rows:")
print(qualifying.head())

print("\nData types:")
print(qualifying.dtypes)

print("\nBasic info:")
print(qualifying.info())

print("\nMissing values:")
print(qualifying.isnull().sum())

=== QUALIFYING DATA EXPLORATION ===

Column names:
['qualifyId', 'raceId', 'driverId', 'constructorId', 'number', 'position', 'q1', 'q2', 'q3']

First few rows:
   qualifyId  raceId  driverId  constructorId  number  position        q1  \
0          1      18         1              1      22         1  1:26.572   
1          2      18         9              2       4         2  1:26.103   
2          3      18         5              1      23         3  1:25.664   
3          4      18        13              6       2         4  1:25.994   
4          5      18         2              2       3         5  1:25.960   

         q2        q3  
0  1:25.187  1:26.714  
1  1:25.315  1:26.869  
2  1:25.452  1:27.079  
3  1:25.691  1:27.178  
4  1:25.518  1:27.236  

Data types:
qualifyId         int64
raceId            int64
driverId          int64
constructorId     int64
number            int64
position          int64
q1               object
q2               object
q3               object
dty

In [4]:
# Let's understand the qualifying time structure
print("=== QUALIFYING TIMES ANALYSIS ===")

# Check what the time data looks like
print("\nSample Q1, Q2, Q3 times:")
print(qualifying[['q1', 'q2', 'q3']].head(10))

# Check for missing times (some drivers don't make it to Q2/Q3)
print("\nMissing times by session:")
print(f"Q1 missing: {qualifying['q1'].isnull().sum()}")
print(f"Q2 missing: {qualifying['q2'].isnull().sum()}")
print(f"Q3 missing: {qualifying['q3'].isnull().sum()}")

# Check the time format
print("\nTime format examples:")
print(qualifying[qualifying['q1'].notna()]['q1'].head())

=== QUALIFYING TIMES ANALYSIS ===

Sample Q1, Q2, Q3 times:
         q1        q2        q3
0  1:26.572  1:25.187  1:26.714
1  1:26.103  1:25.315  1:26.869
2  1:25.664  1:25.452  1:27.079
3  1:25.994  1:25.691  1:27.178
4  1:25.960  1:25.518  1:27.236
5  1:26.427  1:26.101  1:28.527
6  1:26.295  1:26.059  1:28.687
7  1:26.381  1:26.063  1:29.041
8  1:26.919  1:26.164  1:29.593
9  1:26.702  1:25.842        \N

Missing times by session:
Q1 missing: 0
Q2 missing: 0
Q3 missing: 0

Time format examples:
0    1:26.572
1    1:26.103
2    1:25.664
3    1:25.994
4    1:25.960
Name: q1, dtype: object


In [5]:
# Let's connect qualifying to races to get track and date info
print("=== CONNECTING QUALIFYING TO RACES ===")

# Merge qualifying with race info
qualifying_with_races = qualifying.merge(races, on='raceId', how='left')

print("\nColumns after merging with races:")
print(qualifying_with_races.columns.tolist())

print("\nSample data with race info:")
print(qualifying_with_races[['driverId', 'position', 'q1', 'name', 'year', 'circuitId']].head())

# Check date range
print(f"\nDate range: {qualifying_with_races['year'].min()} to {qualifying_with_races['year'].max()}")
print(f"Total qualifying sessions: {len(qualifying_with_races)}")
print(f"Unique races: {qualifying_with_races['raceId'].nunique()}")
print(f"Unique circuits: {qualifying_with_races['circuitId'].nunique()}")

=== CONNECTING QUALIFYING TO RACES ===

Columns after merging with races:
['qualifyId', 'raceId', 'driverId', 'constructorId', 'number', 'position', 'q1', 'q2', 'q3', 'year', 'round', 'circuitId', 'name', 'date', 'time', 'url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time']

Sample data with race info:
   driverId  position        q1                   name  year  circuitId
0         1         1  1:26.572  Australian Grand Prix  2008          1
1         9         2  1:26.103  Australian Grand Prix  2008          1
2         5         3  1:25.664  Australian Grand Prix  2008          1
3        13         4  1:25.994  Australian Grand Prix  2008          1
4         2         5  1:25.960  Australian Grand Prix  2008          1

Date range: 1994 to 2025
Total qualifying sessions: 10774
Unique races: 508
Unique circuits: 43


In [6]:
# Function to convert qualifying time strings to seconds
def time_to_seconds(time_str):
    """Convert time string like '1:23.456' to seconds (83.456)"""
    if pd.isna(time_str) or time_str == '\\N':
        return None

    try:
        # Handle format like '1:23.456'
        if ':' in str(time_str):
            parts = str(time_str).split(':')
            minutes = int(parts[0])
            seconds = float(parts[1])
            return minutes * 60 + seconds
        else:
            # Handle format like '83.456' (already in seconds)
            return float(time_str)
    except:
        return None

# Apply conversion to all qualifying times
print("Converting qualifying times to seconds...")

qualifying_processed = qualifying.copy()
qualifying_processed['q1_seconds'] = qualifying_processed['q1'].apply(time_to_seconds)
qualifying_processed['q2_seconds'] = qualifying_processed['q2'].apply(time_to_seconds)
qualifying_processed['q3_seconds'] = qualifying_processed['q3'].apply(time_to_seconds)

print("\nSample converted times:")
print(qualifying_processed[['q1', 'q1_seconds', 'q2', 'q2_seconds', 'q3', 'q3_seconds']].head())

print("\nMissing times after conversion:")
print(f"Q1: {qualifying_processed['q1_seconds'].isnull().sum()}")
print(f"Q2: {qualifying_processed['q2_seconds'].isnull().sum()}")
print(f"Q3: {qualifying_processed['q3_seconds'].isnull().sum()}")

Converting qualifying times to seconds...

Sample converted times:
         q1  q1_seconds        q2  q2_seconds        q3  q3_seconds
0  1:26.572      86.572  1:25.187      85.187  1:26.714      86.714
1  1:26.103      86.103  1:25.315      85.315  1:26.869      86.869
2  1:25.664      85.664  1:25.452      85.452  1:27.079      87.079
3  1:25.994      85.994  1:25.691      85.691  1:27.178      87.178
4  1:25.960      85.960  1:25.518      85.518  1:27.236      87.236

Missing times after conversion:
Q1: 158
Q2: 4722
Q3: 7007


In [7]:
# Create the target variable: best qualifying time
# This will be what we predict for each driver

def get_best_qualifying_time(row):
    """Get the best (fastest) qualifying time from Q1, Q2, Q3"""
    times = [row['q1_seconds'], row['q2_seconds'], row['q3_seconds']]
    valid_times = [t for t in times if pd.notna(t)]

    if valid_times:
        return min(valid_times)  # Fastest time
    else:
        return None

# Create target variable
qualifying_processed['best_time'] = qualifying_processed.apply(get_best_qualifying_time, axis=1)

print("=== TARGET VARIABLE CREATED ===")
print("\nSample data with best times:")
print(qualifying_processed[['position', 'q1_seconds', 'q2_seconds', 'q3_seconds', 'best_time']].head(10))

print(f"\nTotal records with valid best times: {qualifying_processed['best_time'].notna().sum()}")
print(f"Records missing best times: {qualifying_processed['best_time'].isna().sum()}")

# Verify that best times correlate with positions
print("\nCorrelation between best_time and position (should be positive):")
corr = qualifying_processed[['best_time', 'position']].corr()
print(f"Correlation: {corr.loc['best_time', 'position']:.3f}")

=== TARGET VARIABLE CREATED ===

Sample data with best times:
   position  q1_seconds  q2_seconds  q3_seconds  best_time
0         1      86.572      85.187      86.714     85.187
1         2      86.103      85.315      86.869     85.315
2         3      85.664      85.452      87.079     85.452
3         4      85.994      85.691      87.178     85.691
4         5      85.960      85.518      87.236     85.518
5         6      86.427      86.101      88.527     86.101
6         7      86.295      86.059      88.687     86.059
7         8      86.381      86.063      89.041     86.063
8         9      86.919      86.164      89.593     86.164
9        10      86.702      85.842         NaN     85.842

Total records with valid best times: 10616
Records missing best times: 158

Correlation between best_time and position (should be positive):
Correlation: 0.127


In [8]:
# Let's merge all our data together and start feature engineering
print("=== BUILDING FEATURE DATASET ===")

# Merge qualifying with races and drivers
feature_data = qualifying_processed.merge(races, on='raceId', how='left')
feature_data = feature_data.merge(drivers, on='driverId', how='left')
feature_data = feature_data.merge(constructors, on='constructorId', how='left')

# Only keep records with valid best times (our target)
feature_data = feature_data[feature_data['best_time'].notna()].copy()

print(f"Total records for modeling: {len(feature_data)}")
print(f"Date range: {feature_data['year'].min()} to {feature_data['year'].max()}")
print(f"Unique drivers: {feature_data['driverId'].nunique()}")
print(f"Unique circuits: {feature_data['circuitId'].nunique()}")

# Check what we have to work with
print("\nKey columns for features:")
key_cols = ['year', 'circuitId', 'driverId', 'constructorId', 'best_time', 'position']
print(feature_data[key_cols].head())

=== BUILDING FEATURE DATASET ===
Total records for modeling: 10616
Date range: 1994 to 2025
Unique drivers: 175
Unique circuits: 43

Key columns for features:
   year  circuitId  driverId  constructorId  best_time  position
0  2008          1         1              1     85.187         1
1  2008          1         9              2     85.315         2
2  2008          1         5              1     85.452         3
3  2008          1        13              6     85.691         4
4  2008          1         2              2     85.518         5


In [9]:
# Now let's create your "relative speed" features!
# Feature 1: Driver's historical performance at each track

def create_driver_track_history(df):
    """Create features based on driver's historical performance at each track"""

    # Sort by year to ensure we only use past data
    df_sorted = df.sort_values(['year', 'raceId']).copy()

    # Initialize feature columns
    df_sorted['driver_track_avg_time'] = None
    df_sorted['driver_track_best_time'] = None
    df_sorted['driver_track_appearances'] = 0

    # For each row, calculate historical stats using only PREVIOUS races
    for idx, row in df_sorted.iterrows():
        # Get this driver's previous races at this track
        mask = (
            (df_sorted['driverId'] == row['driverId']) &
            (df_sorted['circuitId'] == row['circuitId']) &
            (df_sorted['year'] < row['year'])  # Only use past data!
        )

        historical_data = df_sorted[mask]

        if len(historical_data) > 0:
            df_sorted.at[idx, 'driver_track_avg_time'] = historical_data['best_time'].mean()
            df_sorted.at[idx, 'driver_track_best_time'] = historical_data['best_time'].min()
            df_sorted.at[idx, 'driver_track_appearances'] = len(historical_data)

    return df_sorted

print("Creating driver track history features...")
print("This might take a moment with 10k+ records...")

# Start with a smaller sample to test (recent years only)
recent_data = feature_data[feature_data['year'] >= 2015].copy()
print(f"Testing with {len(recent_data)} recent records (2015+)")

recent_with_features = create_driver_track_history(recent_data)

print("\nSample of new features:")
feature_cols = ['year', 'driverId', 'circuitId', 'best_time', 'driver_track_avg_time', 'driver_track_appearances']
print(recent_with_features[feature_cols].head(10))

Creating driver track history features...
This might take a moment with 10k+ records...
Testing with 4439 recent records (2015+)

Sample of new features:
      year  driverId  circuitId  best_time driver_track_avg_time  \
6286  2015         1          1     86.327                  None   
6287  2015         3          1     86.921                  None   
6288  2015        13          1     87.718                  None   
6289  2015        20          1     87.742                  None   
6290  2015         8          1     87.790                  None   
6291  2015       822          1     87.796                  None   
6292  2015       817          1     88.329                  None   
6293  2015       832          1     88.510                  None   
6294  2015       154          1     88.560                  None   
6295  2015       813          1     88.726                  None   

      driver_track_appearances  
6286                         0  
6287                         0 

In [10]:
# Let's understand WHY all the historical features are None
# This is a great learning opportunity!

print("=== DEBUGGING THE HISTORICAL FEATURES ===")

# Let's look at one specific driver and track combination
# Driver ID 1 at Circuit ID 1 (let's see who this is)
print("\nWho is driver ID 1?")
driver_info = recent_data[recent_data['driverId'] == 1][['driverId', 'forename', 'surname']].iloc[0]
print(f"Driver 1: {driver_info['forename']} {driver_info['surname']}")

print("\nWhat is circuit ID 1?")
circuit_info = recent_data[recent_data['circuitId'] == 1][['circuitId', 'name_y']].iloc[0]
print(f"Circuit 1: {circuit_info['name_y']}")

# Now let's see this driver's history at this track
print(f"\n=== {driver_info['forename']} {driver_info['surname']} at {circuit_info['name_y']} ===")
driver_at_track = recent_data[
    (recent_data['driverId'] == 1) &
    (recent_data['circuitId'] == 1)
].sort_values('year')

print("All appearances:")
print(driver_at_track[['year', 'best_time', 'position']].to_string())

=== DEBUGGING THE HISTORICAL FEATURES ===

Who is driver ID 1?
Driver 1: Lewis Hamilton

What is circuit ID 1?
Circuit 1: Mercedes

=== Lewis Hamilton at Mercedes ===
All appearances:
       year  best_time  position
6286   2015     86.327         1
6660   2016     83.837         1
7117   2017     82.188         1
7516   2018     81.164         1
7936   2019     80.486         1
9179   2022     78.825         5
9617   2023     77.104         3
10065  2024     76.960        11
10501  2025     75.919         8


In [ ]:
# Set up your training and prediction strategy
print("=== SETTING UP TRAINING/PREDICTION SPLIT ===")

# Training data: 1995-2024
training_data = feature_data[feature_data['year'] <= 2024].copy()

# Prediction target: 2025 races
prediction_data = feature_data[feature_data['year'] == 2025].copy()

print(f"Training data: {len(training_data)} records ({training_data['year'].min()}-{training_data['year'].max()})")
print(f"Prediction data: {len(prediction_data)} records (2025 races)")

print(f"\nTraining data covers {training_data['driverId'].nunique()} unique drivers")
print(f"2025 data has {prediction_data['driverId'].nunique()} unique drivers")

# Key question: Which 2025 drivers are "new" (no historical data)?
training_drivers = set(training_data['driverId'].unique())
prediction_drivers = set(prediction_data['driverId'].unique())

new_drivers_2025 = prediction_drivers - training_drivers
experienced_drivers_2025 = prediction_drivers & training_drivers

print(f"\n=== DRIVER ANALYSIS ===")
print(f"Experienced drivers in 2025: {len(experienced_drivers_2025)}")
print(f"New drivers in 2025: {len(new_drivers_2025)}")

if new_drivers_2025:
    print("\nNew drivers (no historical data):")
    new_driver_names = prediction_data[prediction_data['driverId'].isin(new_drivers_2025)][['driverId', 'forename', 'surname']].drop_duplicates()
    for _, driver in new_driver_names.iterrows():
        print(f"  - {driver['forename']} {driver['surname']} (ID: {driver['driverId']})")